# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

In [2]:
name = "Vera"
f"My name is {name}"

f"{name:>10}"

'      Vera'

In [3]:
for k, v in obj.items():
    print(f"{k:10} = '{v}'")

home_page  = 'https://github.com/pypa/sampleproject'
keywords   = 'sample setuptools development'
license    = 'MIT'


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [6]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

In [7]:
patt = re.compile(r"(?:ПМ)?\s?\d+-\d", re.I)
for item in obj:
    print(f"{item:30}", patt.findall(item))

Евгения гр.ПМ19-1              ['ПМ19-1']
Илья пм 20-4                   ['пм 20-4']
Анна 20-3                      [' 20-3']


In [8]:
obj.str.findall(patt).map(len)

0    1
1    1
2    1
dtype: int64

3. Разбейте текст формулировки задачи 2 на слова.

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\224956\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [10]:
text = "Написать регулярное выражение,которое позволит найти номера групп студентов."
# text.split()
nltk.word_tokenize(text)

['Написать',
 'регулярное',
 'выражение',
 ',',
 'которое',
 'позволит',
 'найти',
 'номера',
 'групп',
 'студентов',
 '.']

In [53]:
tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
tokenizer.tokenize(text)

['Написать',
 'регулярное',
 'выражение',
 'которое',
 'позволит',
 'найти',
 'номера',
 'групп',
 'студентов']

In [54]:
nltk.sent_tokenize("Написать регулярное выражение. Оно позволит найти номера")

['Написать регулярное выражение.', 'Оно позволит найти номера']

## Лабораторная работа 6

In [5]:
import pandas as pd
import re
from bs4 import BeautifulSoup

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [8]:
recipes = pd.read_csv('recipes_sample.csv')

In [9]:
random_recipes = recipes[['id', 'minutes']].sample(5, axis=0)
maxlenid = len(str(random_recipes.id.max()))
maxlenminutes = len(str(random_recipes.minutes.max()))

In [10]:
print(f'|{random_recipes.columns[0]:^{maxlenid+8}}|{random_recipes.columns[1]:^{maxlenminutes+8}}|')

print('|' + (maxlenid+maxlenminutes+17)*'-' + '|')

for recipe in range(len(random_recipes)):
    print(f'|{random_recipes.iloc[recipe].id:^{maxlenid+8}}|{random_recipes.iloc[recipe].minutes:^{maxlenminutes+8}}|')

|      id      |  minutes  |
|--------------------------|
|    517573    |    40     |
|    198028    |    195    |
|    45116     |    90     |
|    215579    |     2     |
|    278538    |    50     |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [21]:
with open(
    "steps_sample.xml",
    "r",
    encoding="utf-8"
) as fp:
    xml = BeautifulSoup(fp)

type(xml)

bs4.BeautifulSoup

In [22]:
recipesxml = xml.find_all('recipe')

In [23]:
def show_info(name, steps, minutes, author_id):

    steps_str = ''
    for i in range(len(steps)):
        steps_str += f'{i+1}. {steps[i].capitalize()}\n'

    return_str = f'"{name.title()}"\n\n' + \
    steps_str + \
    f'----------\nАвтор: {author_id}\nСреднее время приготовления: {minutes} минут\n'
    
    return return_str

In [24]:
recipes_dict = {}
for recipe in recipesxml:
    recipes_dict[int(recipe.find('id').text)] = {'steps': [step.text for step in recipe.find_all('step')]}
for index in recipes.index:
    recipes_dict[recipes['id'][index]].update({
      'name': recipes['name'][index],
      'author_id': recipes['contributor_id'][index],
      'minutes': recipes['minutes'][index]
  })

In [25]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

In [31]:
print(show_info(**recipes_dict[170895]))

"Leeks And Parsnips  Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addi

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [26]:
patt = re.compile(r"(\d+\s(?:(?:hours?)|(?:minutes?)))", re.I)

for step in recipes_dict[25082]['steps']:
    if patt.findall(step) != []:
        print(patt.findall(step))

['20 minutes']
['10 minutes']
['2 hours']
['10 minutes']
['20 minutes', '30 minutes']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [27]:
patt = re.compile(r"^this[\w ]+,\s?but")
                  
reg_recipes = recipes['description'].str.match(patt)
reg_recipes_true = reg_recipes[reg_recipes == True].index
len(reg_recipes_true)

134

In [28]:

recipes['description'][reg_recipes_true][:3].values

array(["this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing.",
       "this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.",
       'this is kind of similar to some of the other versions out there, but it is the best and easiest i have found'],
      dtype=object)

5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [29]:
def replace(s):
    return ''.join(s.group(0).split(' '))

patt = re.compile(r"\d+\s/\s\d+")
steps_new = []

for step in recipes_dict[72367]['steps']:
    step_all = patt.findall(step)
    for s in step_all:
        step = re.sub(patt, replace, step)
    steps_new.append(step)

recipes_dict[72367]['steps'] = steps_new
print(recipes_dict[72367]['steps'])

['mix butter , flour , 1/3 c', 'sugar and 1-1/4 t', 'vanilla', 'press into greased 9" springform pan', 'mix cream cheese , 1/4 c', 'sugar , eggs and 1/2 t', 'vanilla beating until fluffy', 'pour over dough', 'combine apples , 1/3 c', 'sugar and cinnamon', 'arrange on top of cream cheese mixture and sprinkle with almonds', 'bake at 350 for 45-55 minutes , or until tester comes out clean']


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [33]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [34]:
tokenizer = nltk.tokenize.RegexpTokenizer(r"[a-zA-Z]+'?[a-zA-Z]*")

word_list = []

for recipe in recipes_dict:
    word_list.extend(set(tokenizer.tokenize(','.join(recipes_dict[recipe]['steps']))))

word_list = set(word_list)
word_list_lower = []
for word in word_list:
    word_list_lower.append(word.lower())

len(set(word_list_lower))

15507

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [35]:
recipes_copy = recipes.copy()
description_sent = []
for recipe in recipes_copy['description']:
    description_sent.append(nltk.sent_tokenize(str(recipe)))
recipes_copy['description_sent'] = description_sent
recipes_copy['description_sent'].head()

0    [an original recipe created by chef scott mesk...
1    [my children and their friends ask for my home...
2            [these were so go, it surprised even me.]
3    [my sister-in-law made these for us at a famil...
4    [i think a fondue is a very romantic casual di...
Name: description_sent, dtype: object

In [37]:
recipes_copy['description_len'] = recipes_copy['description_sent'].str.len()
longest = recipes_copy['description_len'][recipes_copy['description_len'].isin(sorted(recipes_copy['description_len'], reverse = True)[:4])]

In [38]:
recipes.iloc[longest.sort_values(ascending = False).index]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes a...,NaN
481,alligator claws avocado fritters with chipot...,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the gr...,9.0
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding l...,NaN
16296,little bunny foo foo cake carrot cake with c...,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a mil...,NaN


In [39]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [40]:
tokenizer = nltk.tokenize.RegexpTokenizer((r"[a-zA-Z]+'?[a-zA-Z]*"))

sent = 'I omitted the raspberries and added strawberries instead'
words = tokenizer.tokenize(sent)
nltk.pos_tag(words)


[('I', 'PRP'),
 ('omitted', 'VBD'),
 ('the', 'DT'),
 ('raspberries', 'NNS'),
 ('and', 'CC'),
 ('added', 'VBD'),
 ('strawberries', 'NNS'),
 ('instead', 'RB')]

In [41]:
def part_of_speech(sent):
    parts_str = ''
    words_str = ''
    tokenizer = nltk.tokenize.RegexpTokenizer(r"[a-zA-Z]+")
    words = tokenizer.tokenize(sent)
    words_parts = nltk.pos_tag(words)
    print(words)
    for word in words_parts:
        parts_str = parts_str + f'{word[1]:^{len(word[0])}} '
        words_str = words_str + f'{word[0]:^{len(word[1])}} '
    return parts_str + '\n' + words_str
        
print(part_of_speech(sent))

['I', 'omitted', 'the', 'raspberries', 'and', 'added', 'strawberries', 'instead']
PRP   VBD   DT      NNS     CC   VBD      NNS        RB    
 I  omitted the raspberries and added strawberries instead 


In [42]:
test = recipes['name'][recipes['id'] == 241106].values[0]
print(part_of_speech(test))

['eggplant', 'steaks', 'with', 'chickpeas', 'feta', 'cheese', 'and', 'black', 'olives']
   JJ     NNS    IN     NNS    VBP    JJ   CC   JJ    NNS   
eggplant steaks with chickpeas feta cheese and black olives 
